In [55]:


import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
#from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor  #GBM algorithm
from sklearn.metrics import accuracy_score, confusion_matrix
#from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from xgboost.sklearn import XGBRegressor, XGBClassifier


#from vars_utils import create_vars
df = pd.read_csv('./modeling_data/tgs_models.csv')

cutoff_week = 4 
cur_week = 18 
begin_year=2014, 
val_year_cutoff = 2021, 
targ='fav_cover'
#df = pd.read_csv(df, error_bad_lines=True)
df=df.apply(pd.to_numeric, errors='ignore')

df = df.dropna(thresh=df.shape[1]-60)
df = df[df['fav_cover'] != 'push']
df['fav_cover'] = df['fav_cover'].apply(str)



df.replace(np.nan, 0.01, inplace=True)
df.fillna(df.median(), inplace=True)
df.drop_duplicates(inplace=True)


/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [56]:
df['fav_cover']=df['fav_cover'].replace(to_replace = ['cover','no cover'],value = [1,0])

In [57]:
df.replace(np.inf, .01, inplace=True)
df.replace(-np.inf, -.01, inplace=True)
df.replace(np.nan, 0, inplace=True)

df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df.dropna(axis=1, how='all', inplace=True)

df = df[df['year'] >= begin_year]
df = df[df['week'] >= cutoff_week]
test = df[(df['year'] >= val_year_cutoff)|(df['year'] >= 2020)]
test_10 = test[(test['year'] == 2020) & (test['week'] == cur_week)]
test=test[~test.index.isin(test_10.index)]

y_test = test[targ]

data_Model = df[~df.index.isin(test.index)]
data_Model = data_Model[~data_Model.index.isin(test_10.index)]
data_Model = data_Model[(data_Model['week'] >= 4)|(df['year'] < 2020)]
y_train = data_Model[targ]

#y_reg_train = data_Model['spread_favorite']


plyr_game_cols =[x for x in data_Model.columns[data_Model.columns.str.contains('player_game')]]
week_game_cols =[x for x in data_Model.columns[data_Model.columns.str.contains('week_rank')]]
pred_cols = [x for x in data_Model.columns if x not in ['team_id','team_favorite_id','fav_cover', 'home_matchup_id','home_team', 'away_team','team_conference_x','team_conference_y','team_division_x','team_division_y','fav_homeoraway_y','pf_x','pf_y','WLT','outlier','wl_x','wl_y','OU_result','pf','opponent_id_overall_pff_x','opponent_id_overall_pff_y', 'opponent_id_x','opponent_id_overall_pff','wl','WL','home_matchup_id_x','home_matchup_id_y','opponent_id_overall_pff_x','over_under_result','def_pff_prush_x','unique_team_id_x','unique_team_id_y','home_matchup_id']+plyr_game_cols+week_game_cols]


X_train = data_Model[pred_cols]
X_test = test[pred_cols]



/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [58]:
len(X_test), len(y_test)

(420, 420)

In [59]:

pipe_lr = Pipeline([('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
('pca', PCA(n_components=X_train.shape[1]-1)),
('clf', LogisticRegression(penalty='l1', C=.1, solver='liblinear',random_state=42))])

pipe_lr2 = Pipeline([#('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
#('pca', PCA(n_components=X_train.shape[1]-1)),
('clf', LogisticRegression(penalty='l2', C=.1, solver='liblinear',random_state=42))])

pipe_xgb = Pipeline([('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", C=.1, dual=False))),
#('pca', PCA(n_components=X_train.shape[1]-20)),
('clf', XGBClassifier(n_estimators=500, random_state=42))])

pipe_rf= Pipeline([#('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", C=1, dual=False))),
#('pca', PCA(n_components=X_train.shape[1]-50)),
#('clf', svm.SVC(probability=True, C=50, gamma=.1, kernel='rbf', random_state=42))])
('clf', RandomForestClassifier(n_estimators=700, n_jobs=4, random_state=42))])

pipe_gbm= Pipeline([#('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", C=.01, dual=False))),
('clf', XGBClassifier(n_estimators=500, random_state=42))])

pipe_gbm2= Pipeline([#('scl', MinMaxScaler()),
#('feature_selection', SelectFromModel(LinearSVC(penalty="l1", C=.1, dual=False))),
#('pca', PCA(n_components=X_train.shape[1]-25)),
('clf', GradientBoostingClassifier(n_estimators=500, learning_rate=.01, random_state=42))])


# List of pipelines for ease of iteration
pipelines = [pipe_lr, pipe_lr2, pipe_xgb, pipe_rf, pipe_gbm, pipe_gbm2]

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0:'LR', 1:'lr2', 2:'xgb', 3:'rForest', 4:'gbm', 5:'gbm2'}

# Fit the pipelines
for pipe in pipelines:
    print(pipe)
    pipe.fit(X_train, y_train)

# Compare accuracies
for idx, val in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))


# Identify the most accurate model on test data
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
    if val.score(X_test, y_test) > best_acc:
        best_acc = val.score(X_test, y_test)
        best_pipe = val
        best_clf = idx

print('Classifier with best accuracy: %s' % pipe_dict[best_clf])

Pipeline(steps=[('scl', MinMaxScaler()), ('pca', PCA(n_components=49)),
                ('clf',
                 LogisticRegression(C=0.1, penalty='l1', random_state=42,
                                    solver='liblinear'))])
Pipeline(steps=[('clf',
                 LogisticRegression(C=0.1, random_state=42,
                                    solver='liblinear'))])
Pipeline(steps=[('scl', MinMaxScaler()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
           